In [13]:
# Optimizing Model Parameters 优化模型参数
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.models as models

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

In [14]:
# Hyperparameters 超参数 超参数是可调参数，可控制模型优化过程。
# Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [15]:
# Optimization Loop 优化循环 
#设置好超参数后，我们就可以通过优化循环来训练和优化模型。优化循环的每次迭代称为一个纪元。

In [16]:
# Initialize the loss function
# 当遇到一些训练数据时，未经训练的网络很可能无法给出正确答案。损失函数衡量的是获得的结果与目标值的不相似程度，在训练过程中将损失函数最小化。为了计算损失，我们使用给定数据样本的输入进行预测，并与真实的数据标签值进行比较。
loss_fn = nn.CrossEntropyLoss()

In [17]:
# Optimizer 优化器
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [18]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {
          (100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    loss_fn = nn.CrossEntropyLoss()# 使用交叉熵损失函数


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301204  [   64/60000]
loss: 2.290670  [ 6464/60000]
loss: 2.266337  [12864/60000]
loss: 2.264369  [19264/60000]
loss: 2.250949  [25664/60000]
loss: 2.221542  [32064/60000]
loss: 2.226750  [38464/60000]
loss: 2.192129  [44864/60000]
loss: 2.179458  [51264/60000]
loss: 2.163447  [57664/60000]
Test Error: 
 Accuracy: 46.1%, Avg loss: 2.152936 

Epoch 2
-------------------------------
loss: 2.157668  [   64/60000]
loss: 2.149447  [ 6464/60000]
loss: 2.090186  [12864/60000]
loss: 2.112486  [19264/60000]
loss: 2.073620  [25664/60000]
loss: 2.006741  [32064/60000]
loss: 2.031917  [38464/60000]
loss: 1.953234  [44864/60000]
loss: 1.947244  [51264/60000]
loss: 1.895514  [57664/60000]
Test Error: 
 Accuracy: 61.1%, Avg loss: 1.888183 

Epoch 3
-------------------------------
loss: 1.914075  [   64/60000]
loss: 1.884195  [ 6464/60000]
loss: 1.769040  [12864/60000]
loss: 1.815733  [19264/60000]
loss: 1.729667  [25664/60000]
loss: 1.670044  [32064/600

In [19]:
# save model
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model.state_dict(), 'model_weights.pth')
model = models.vgg16()  # we do not specify ``weights``, i.e. create untrained model
model.load_state_dict(torch.load('model_weights.pth', weights_only=True))
model.eval()
# be sure to call model.eval() method before inferencing to set the dropout and batch normalization layers to evaluation mode. Failing to do this will yield inconsistent inference results.


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [21]:
torch.save(model, 'model.pth')
model = torch.load('model.pth', weights_only=False)